----
**Research Question**

## Investigation of changes in **Depressive-Symptoms** influenced by drinking frequency of **Sugar-Sweetened** soft drinks.
----

In [1]:
# importting necessary libraries 

import pandas as pd
import numpy as np
import yaml as yam
from bokeh.plotting import figure,show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10
import holoviews as hv
from bokeh.io import output_notebook

output_notebook()


Loading BokehJS ...

----
## Part 1 : Loading The Data
----

In [2]:
# creating a config file , to read data from

def get_config():
    with open('config.yaml','r') as datasource:
        config = yam.safe_load(datasource)
        return config

config = get_config()

depressive_symptoms = config['depressive_symptoms']
frequency_of_drinking = config['frequency_of_drinking']

# reading data by creating pandas dataframe
depsymp = pd.read_csv(depressive_symptoms)
drinkfreq = pd.read_csv(frequency_of_drinking)



----
## Part 2 : **Data Preparation**
---- 

* Renaming columns

In [3]:
# Renaming the desired columns in order to have a better understanding of the columns name.

def rename(df,col_list) :
    rename = df.rename(columns= col_list)
    df = rename
    return df

depsymp_col_list = {'isced11':'Edu_Level','hlth_pb':'Health_Issue',
            'sex':'Gender','age':'Age_Range','geo':'Country','TIME_PERIOD':'Time_Period',
            'OBS_VALUE':'Obs_Value'}

depsymp = rename(depsymp,depsymp_col_list)


drinkfreq_col_list = {'frequenc':'Drink_Freq','sex':'Gender','isced11':'Edu_Level',
            'age':'Age_Range','geo':'Country','TIME_PERIOD':'Time_Period','OBS_VALUE':'Obs_Value'}

drinkfreq = rename(drinkfreq,drinkfreq_col_list)

* Identifying missing values

In [5]:
# defining a function to return missing values in multiple dataframes,
# output in the form of a dictionary will contain both dataframe and columns 
# with missing values 

# depsymp.isna().sum()
# depsymp.isnull().sum()


def find_missing_values(df_list):
    missing_values = []
    for i,df in enumerate(df_list):

        df_missing = df.isna().sum().to_dict()
        df_missing = {k: v for k,v in df_missing.items() if v > 0}
        missing_values.append(f'dataframe {i+1} = {df_missing}')
                
    return missing_values


missing_values = find_missing_values([depsymp,drinkfreq])

# print(missing_values)

* Identifying unreliable observations

In [6]:
# Flag columns indicate observations which are not reliable.
# Below, identification of the specific value for unreliable abservations and the length of rows containing 
# them where inspected.

print(depsymp['OBS_FLAG'].unique())
print(drinkfreq['OBS_FLAG'].unique())
print(len(depsymp[depsymp['OBS_FLAG'] == 'u']))
print(len(drinkfreq[drinkfreq['OBS_FLAG'] == 'u']))

[nan 'u']
[nan 'u']
36
27


* Removing unreliable observations and columns with missing values

In [7]:
# defining a function that takes in list of dataframes and returns 
# cleaned dataframes (without columns containing missing values, and rows with unreliable observations)

def drop_missing_values(dataframes):
    cleaned_dataframes = []
    for df in dataframes:
        if 'OBS_FLAG' in df.columns :
            df = df[df['OBS_FLAG'] != 'u']
        df_dropped = df.dropna(axis=1)
        cleaned_dataframes.append(df_dropped)
    return cleaned_dataframes

list = [depsymp,drinkfreq]
cleaned_dataframes = drop_missing_values(list)

depsymp = cleaned_dataframes[0]
drinkfreq = cleaned_dataframes[1]
# drinkfreq



* Dropping irrelevant columns 

In [8]:
# dropping irrelevent columns using if statements to 
# avoid code execution more than once

if 'DATAFLOW' in depsymp.columns :
    depsymp = depsymp.drop(depsymp.columns[[0,1,2,3,4]],axis=1)

if 'DATAFLOW' in drinkfreq.columns :
    drinkfreq = drinkfreq.drop(drinkfreq.columns[[0,1,2,3,7]],axis=1)
    
# print(depsymp)
# print(drinkfreq) 


In [9]:
# checking for duplicated values in both dataframes.

# df.duplicated()
# df.drop_duplicates()

def check_for_duplicated(df_list) :
    for df in df_list :
        if df.duplicated().any() :
            return True
        else :
            return False

duplicated = check_for_duplicated([depsymp,drinkfreq])
# print(duplicated)


----
## Part 3 : Merging Dataframes 
----

In [10]:
# depsymp = depsymp[depsymp['TIME_PERIOD']== 2019]
# print(depsymp)

In [11]:
# using pandas merge method we are going to combine two dataframes 
# into one with values of interest for further analysing.

df = pd.merge(drinkfreq,depsymp,on=['Country','Time_Period','Age_Range','Gender'],how='inner')

# Since we are going to compare frequency of drinking sweetened drinks\
# with depressive modes , we will define a function to enable us to call for specific 
# depressive mode(out of three) each time in one of the age ranges and see\
# how level of drinking(4 different types of drinking habits) would influences 
# the specific depressive mode.

def get_data(Drink_Freq,Health_Isuue,Age_Range) :

    data = df.loc[
        (df['Drink_Freq'] == Drink_Freq)&(df['Health_Issue'] == Health_Isuue)&
        (df['Age_Range'] == Age_Range)&(df['Gender'] == 'T')
        ]
    data = data.reset_index(drop=True)
    return data

df1 = get_data('1-3W','DPR','Y15-24')
df2 = get_data('4-6W','DPR','Y15-24')
df3 = get_data('GE1D','DPR','Y15-24')
df4 = get_data('NVR_OCC','DPR','Y15-24')

df.head()

,Drink_Freq,Gender,Age_Range,Country,Time_Period,Obs_Value_x,Health_Issue,Obs_Value_y
0,1-3W,F,Y15-24,AT,2019,26.6,DPR,5.4
1,1-3W,F,Y15-24,AT,2019,26.6,DPR_MJR,2.1
2,1-3W,F,Y15-24,AT,2019,26.6,DPR_OTH,3.3
3,4-6W,F,Y15-24,AT,2019,9.5,DPR,5.4
4,4-6W,F,Y15-24,AT,2019,9.5,DPR_MJR,2.1


----

## Part 4 : Smooth The Data

In [12]:
# defining a function that takes in a list of dataframes and retunrs a list of 
# smoothed data frames. The built in method '.rolling().mean()' was used.

def smooth_dataframes(dataframes,window_size) :
    smoothed_dataframes = []
    for df in dataframes :
        df = df.copy()
        df['Obs_Value_x'] = df['Obs_Value_x'].rolling(window=window_size).mean()
        smoothed_dataframes.append(df)
    return smoothed_dataframes

dataframes = smooth_dataframes([df1,df2,df3,df4],10)
# dataframes[1]

----

## Part 5 : Data Visualization 

In [13]:
# plotting was done using bokeh library

p = figure(x_range=df1['Country'].tolist(), plot_width=600, plot_height=400)

palette = Category10[5]
dataframes = smooth_dataframes([df1,df2,df3,df4],3)

for i,df in enumerate(dataframes) :
    source = ColumnDataSource(df)
    p.line(x='Country',y='Obs_Value_x',source=source,color=palette[i],legend_label=f'df {i+1}')

p.line(x='Country',y='Obs_Value_y',source=df1,color=palette[i],legend_label='DPR')

# p.line(x='geo', y='OBS_VALUE_x', source=source, line_width=2, legend_label='OBS_VALUE_x')
# p.line(x='geo', y='OBS_VALUE_y', source=source, line_width=2, legend_label='OBS_VALUE_y', line_color='red')

p.xaxis.axis_label = 'Country'
p.yaxis.axis_label = 'Obs_Value'

p.legend.location = 'top_left'

show(p)